In [1]:
import pandas as pd 
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

In [2]:
df = pd.read_csv("../raw_data/df_energy_weather.csv", index_col=[0])

In [3]:
#df = df[['Consommation (MW)','T2M', 'T2MDEW', 'T2MWET', 'TS', 'T2M_RANGE','T2M_MAX', 'T2M_MIN', 'QV2M', 'RH2M', 'PRECTOTCORR', 'PS', 'WS10M','WS50M']]

In [4]:
df = df[['Consommation (MW)','T2M']]

In [5]:
#test

#from sklearn.preprocessing import MinMaxScaler
#scaler = MinMaxScaler(feature_range = (0, 1))

#df = scaler.fit_transform(df)
#df

In [6]:
df.isnull().sum()

Consommation (MW)    0
T2M                  0
dtype: int64

In [7]:
def subsample_sequence(df, length):

    # $CHALLENGIFY_BEGIN
    last_possible = df.shape[0] - length
    
    random_start = np.random.randint(0, last_possible)
    df_sample = df[random_start: random_start+length]
    # $CHALLENGIFY_END
    return df_sample


In [8]:
def split_subsample_sequence(df, length_x, ecart , length_y ):
    '''Create one single random (X,y) pair'''
    # $CHALLENGIFY_BEGIN
    length = length_x + ecart + length_y
    df_subsample = subsample_sequence(df, length)
    y_sample = df_subsample.iloc[length_x + ecart:length]['Consommation (MW)']
    
    X_sample = df_subsample[0:length_x]#.drop(['Consommation (MW)'], axis=1)
    #X_sample = X_sample.drop(['Consommation (MW)'], axis=1)
    #X_sample = X_sample.drop(columns=['Consommation (MW)'])
    X_sample = X_sample.values
    # $CHALLENGIFY_END
    return np.array(X_sample), np.array(y_sample)

In [9]:
def get_X_y(df, n_sequences, length_x, ecart , length_y):
    '''Return a list of samples (X, y)'''
    # $CHALLENGIFY_BEGIN
    
    
    X, y = [], []

    for i in range(n_sequences):
        (xi, yi) = split_subsample_sequence(df, length_x, ecart , length_y)
        X.append(xi)
        y.append(yi)
        
    X = np.array(X)
    y = np.array(y)
    # $CHALLENGIFY_END
    
    #X = np.reshape(X, (X.shape[0], X.shape[1], 1)
    #y = np.reshape(y, (y.shape[0], y.shape[1], 1)
                   
    return X, y

In [10]:
#features_set, labels = np.array(features_set), np.array(labels)
#features_set

In [11]:
def creation_trains_tests(df, n_sequences, length_x, ecart, length_y, pourcent_test):
    len_ = int(pourcent_test*df.shape[0])
    df_train = df[:len_]
    df_test = df[len_:]
    X_train, y_train = get_X_y(df_train, n_sequences, length_x, ecart, length_y)
    X_test, y_test = get_X_y(df_test, n_sequences, length_x, ecart, length_y)
    print(X_train.shape)
    print(y_train.shape)
    print(X_test.shape)
    print(y_test.shape)
    return X_train, y_train, X_test, y_test

In [12]:
n_sequences = 100
length_x = 350
ecart = 0
length_y = 10
pourcent_test = 0.7

In [13]:
X_train, y_train, X_test, y_test = creation_trains_tests(df, n_sequences, length_x, ecart, length_y, pourcent_test)

(100, 350, 2)
(100, 10)
(100, 350, 2)
(100, 10)


In [14]:
X_train[2][0]
#X_train[0][:5]

array([1.92849e+05, 1.80500e+01])

In [15]:
#X_train[4][:5]

In [16]:
# Revoir la liste des imports nécessaires
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import optimizers, metrics
from tensorflow.keras.layers.experimental.preprocessing import Normalization
from tensorflow.keras.callbacks import EarlyStopping


In [17]:


model_stacked = models.Sequential()

model_stacked.add(layers.LSTM(10, return_sequences=True , input_shape = (X_train.shape[1],1))) 
model_stacked.add(layers.LSTM(6, return_sequences=False))
model_stacked.add(layers.Dense(y_train.shape[1], activation='linear'))

model_stacked.summary()




2022-06-16 16:09:30.685126: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 350, 10)           480       
_________________________________________________________________
lstm_1 (LSTM)                (None, 6)                 408       
_________________________________________________________________
dense (Dense)                (None, 10)                70        
Total params: 958
Trainable params: 958
Non-trainable params: 0
_________________________________________________________________


In [18]:
X_train[0][0]

array([2.02153e+05, 1.80800e+01])

In [19]:
model_stacked.compile(loss='mse', optimizer='adam')

es = EarlyStopping(monitor='val_loss', verbose=1, patience=5, restore_best_weights=True)

model_stacked.fit(X_train, y_train, epochs=10, batch_size=32, callbacks=[es], verbose=1)

Epoch 1/10


2022-06-16 16:09:30.962475: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)


ValueError: in user code:

    /Users/thomasbordier/.pyenv/versions/3.8.13/envs/forecast_energy_consumption/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:855 train_function  *
        return step_function(self, iterator)
    /Users/thomasbordier/.pyenv/versions/3.8.13/envs/forecast_energy_consumption/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:845 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /Users/thomasbordier/.pyenv/versions/3.8.13/envs/forecast_energy_consumption/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:1285 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /Users/thomasbordier/.pyenv/versions/3.8.13/envs/forecast_energy_consumption/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:2833 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /Users/thomas

In [ ]:
#model_stacked.predict(X_test)

In [ ]:
model.summary()

In [ ]:
normalizer = Normalization()
normalizer.adapt(X_train)

In [ ]:
es = EarlyStopping(monitor='val_loss', verbose=1, patience=5, restore_best_weights=True)

regressor = models.Sequential()

regressor.add(normalizer)
# First LSTM layer with Dropout regularisation
regressor.add(layers.LSTM(units=50, return_sequences=True ,activation='tanh',input_shape=(X_train.shape[1],1)))#input_shape=(X_train.shape[1:]))) #input_shape=(X_train.shape[1],1)))
regressor.add(layers.Dropout(0.2))
# Second LSTM layer
regressor.add(layers.LSTM(units=50, return_sequences=True,activation='tanh'))
regressor.add(layers.Dropout(0.2))
# Third LSTM layer
regressor.add(layers.LSTM(units=50, return_sequences=True,activation='tanh'))
#regressor.add(Dropout(0.2))
# Fourth LSTM layer
regressor.add(layers.LSTM(units=50,activation='tanh'))
#regressor.add(Dropout(0.2))
# The output layer
regressor.add(layers.Dense(units=10,activation = 'linear'))

# Compiling the RNN
regressor.compile(optimizer='rmsprop',loss='mean_squared_error', metrics=tf.keras.metrics.MAPE)
# Fitting to the training set
regressor.fit(X_train,y_train,validation_split=0.2,epochs=100,callbacks=[es], batch_size=None)




In [ ]:
#Epoch 1000/1000
#3/3 [==============================] - 1s 295ms/step - loss: 50108309504.0000 - mean_absolute_percentage_error: 99.9273 - val_loss: 52714774528.0000 - val_mean_absolute_percentage_error: 99.9284

In [ ]:
#regressor.summary()

In [ ]:
y_train.shape

In [ ]:
metric = metrics.MAPE
opt = optimizers.RMSprop(learning_rate=0.005)
normalizer = Normalization()
normalizer.adapt(X_train)

model = models.Sequential()
model.add(normalizer)
model.add(layers.LSTM(20, return_sequences=True, activation='tanh'))
model.add(layers.LSTM(10, activation='tanh'))
model.add(layers.Dense(5, activation='relu'))
model.add(layers.Dense(10, activation='linear'))

model.compile(loss='mse', 
              optimizer=opt, 
              metrics=[metric])

es = EarlyStopping(monitor='val_loss', verbose=1, patience=5, restore_best_weights=True)

history_2 = model.fit(X_train, y_train,
            validation_split=0.3,
            epochs=10, 
            batch_size=32,
            callbacks=[es], verbose=1)